In [4]:
# Import dependencies
import pandas as pd
import random
import datetime as dt

In [5]:
# Jupyter Notebook raises warnings about the data types in a couple columns of the later data sets. These columns get dropped, 
# so the warnings aren't important. This suppresses the warnings as the code runs.

import warnings
warnings.filterwarnings('ignore')

In [6]:
# Create lists of raw data files. The structure of the files changes after January 2021, and need to be processed slightly differently.

raw_data_early = ["raw_data/201809-citibike-tripdata.csv", "raw_data/201810-citibike-tripdata.csv", "raw_data/201811-citibike-tripdata.csv", "raw_data/201812-citibike-tripdata.csv", "raw_data/201901-citibike-tripdata.csv", "raw_data/201902-citibike-tripdata.csv", "raw_data/201903-citibike-tripdata.csv", "raw_data/201904-citibike-tripdata.csv", "raw_data/201905-citibike-tripdata.csv", "raw_data/201906-citibike-tripdata.csv", "raw_data/201907-citibike-tripdata.csv", "raw_data/201908-citibike-tripdata.csv", "raw_data/201909-citibike-tripdata.csv", "raw_data/201910-citibike-tripdata.csv", "raw_data/201911-citibike-tripdata.csv", "raw_data/201912-citibike-tripdata.csv", "raw_data/202001-citibike-tripdata.csv", "raw_data/202002-citibike-tripdata.csv", "raw_data/202003-citibike-tripdata.csv", "raw_data/202004-citibike-tripdata.csv", "raw_data/202005-citibike-tripdata.csv", "raw_data/202006-citibike-tripdata.csv", "raw_data/202007-citibike-tripdata.csv", "raw_data/202008-citibike-tripdata.csv", "raw_data/202009-citibike-tripdata.csv", "raw_data/202010-citibike-tripdata.csv", "raw_data/202011-citibike-tripdata.csv", "raw_data/202012-citibike-tripdata.csv", "raw_data/202101-citibike-tripdata.csv"]

raw_data_later = ["raw_data/202102-citibike-tripdata.csv", "raw_data/202103-citibike-tripdata.csv", "raw_data/202104-citibike-tripdata.csv", "raw_data/202105-citibike-tripdata.csv", "raw_data/202106-citibike-tripdata.csv", "raw_data/202107-citibike-tripdata.csv", "raw_data/202108-citibike-tripdata.csv", "raw_data/202109-citibike-tripdata.csv", "raw_data/202110-citibike-tripdata.csv", "raw_data/202111-citibike-tripdata.csv", "raw_data/202112-citibike-tripdata.csv", "raw_data/202201-citibike-tripdata.csv", "raw_data/202202-citibike-tripdata.csv", "raw_data/202203-citibike-tripdata.csv", "raw_data/202204-citibike-tripdata.csv", "raw_data/202205-citibike-tripdata.csv", "raw_data/202206-citibike-tripdata.csv", "raw_data/202207-citibike-tripdata.csv", "raw_data/202208-citibike-tripdata.csv", "raw_data/202209-citibike-tripdata.csv"]

In [74]:
# This function cleans the data from data files from February, 2021 and later.

def clean_data_later(sample):
    # Drop the columns I'm not interested in
    dropped_columns = sample.drop(columns=["ride_id", "start_station_id", "end_station_id", "rideable_type"])
    # Drop any entries with missing data
    dropped_nas = dropped_columns.dropna()
    # Drop any duplicated entries
    dropped_dups = dropped_nas.drop_duplicates(ignore_index=True)
    # Reindex the cleaned dataframe
    reindexed_df = dropped_dups.reset_index(drop=True)
    
    return reindexed_df

In [7]:
# This function is an adjusted version of my clean_data function. This function has the additional steps of masking the titles
# of the columns to match the columns of the data files after January, 2021. This function is only used for data files from
# January, 2021 and earlier.

def clean_data_early(sample):
    # Mask the column titles so they match with the later data sets
    masked_cols = sample.rename(columns = {"tripduration": "ride_time", "starttime": "started_at", "stoptime": "ended_at", "start station id": "start_station_id", "start station name": "start_station_name", "start station latitude": "start_lat", "start station longitude": "start_lng", "end station id": "end_station_id", "end station name": "end_station_name", "end station latitude": "end_lat", "end station longitude": "end_lng", "usertype":"member_casual"})
    # Mask the values of the column to match later data sets
    masked_cols["member_casual"] = masked_cols["member_casual"].map({"Subscriber": "member", "Customer": "casual"})
    # Drop columns I'm not interested in
    dropped_cols = masked_cols.drop(columns=["start_station_id", "end_station_id", "bikeid", "birth year", "gender"])
    # Rearrange the columns to match the later data sets
    rearranged = dropped_cols[["started_at", "ended_at", "start_station_name", "start_lat", "start_lng", "end_station_name", "end_lat", "end_lng", "member_casual", "ride_time"]]
    # Drop any entries with incomplete data
    dropped_nas = rearranged.dropna()
    # Drop any duplicated entries
    dropped_dups = dropped_nas.drop_duplicates(ignore_index = True)
    # Reindex the dataframe
    reindexed_df = dropped_dups.reset_index(drop = True)
    
    return reindexed_df

In [8]:
# Takes a random sample of 5,000 entries for visualization with Tableau. This keeps the amount of data Tableau has to work 
# with to a more reasonable level, and it keeps the resulting CSV file under GitHub's maximum file size.
def take_random_sample(df):
    # Make a random list of 5,000 numbers from the range of indices
    random_list = random.sample(range(0, len(df)), 5000)
    # Use the list of random numbers to pull out entries with the corresponding index
    sample_df = df[df.index.isin(random_list)]
    # Reset the index
    sample_df = sample_df.reset_index(drop=True)
    
    return sample_df

In [9]:
# Round the geographic coordinates of each entry to 3 decimal points. This helps trips at the same stations to get grouped 
#together.

def round_coords(df):
    df["start_lat"] = df["start_lat"].round(3)
    df["start_lng"] = df["start_lng"].round(3)
    df["end_lat"] = df["end_lat"].round(3)
    df["end_lng"] = df["end_lng"].round(3)
    
    return df

In [53]:
def extract_month_year_early(raw_data):
    first_date = raw_data["starttime"][0].split("-")
    month_year = first_date[1] + "-" + first_date[0]
    return month_year

In [68]:
def extract_month_year_later(raw_data):
    first_date = raw_data["started_at"][0].split("-")
    month_year = first_date[1] + "-" + first_date[0]

In [70]:
def total_ridership(raw_data):
    # Count the number of riders in the full, cleaned data set
    ridership = len(raw_data)
    
    return ridership

In [57]:
def process_early_better(csv_path):
    #read in csv
    raw_data_df = pd.read_csv(csv_path)
    # sample
    sample = take_random_sample(raw_data_df)
    #clean
    cleaned_data = clean_data_early(sample)
    # round
    rounded_data = round_coords(cleaned_data)
    # add month_year
    rounded_data.insert(loc = 10, column = "month_year", value = extract_month_year_early(raw_data_df))
    # add total_ridership
    rounded_data.insert(loc = 11, column = "total_ridership", value = total_ridership(raw_data_df))
        
    return rounded_data

In [83]:
# This calculates how long each trip took in seconds and adds it to the dataframe. This is only needed for data *after*
# January 2021.

def calculate_ride_time(df):
    ride_time = []

    for event in range(0, len(df)):
        # Convert started_at to datetime object, save it
        started_dt = dt.datetime.strptime(df["started_at"][event], "%Y-%m-%d %H:%M:%S")
        # Convert ended_at to datetime object, save it
        ended_dt = dt.datetime.strptime(df["ended_at"][event], "%Y-%m-%d %H:%M:%S")
        # Calculate elapsed time in seconds
        time_change = ended_dt - started_dt
        elapsed_seconds = time_change.seconds        
        # Append string to series
        ride_time.append(elapsed_seconds)
        
    df["ride_time"] = ride_time
    
    return df

In [92]:
def process_later_better(csv_path):
    #read
    raw_data_df = pd.read_csv(csv_path)
    #sample
    sample = take_random_sample(raw_data_df)
    #clean
    cleaned_data = clean_data_later(sample)
    #round
    rounded_data = round_coords(cleaned_data)
    #add ride_time
    with_ride_time = calculate_ride_time(rounded_data)
    #add month_year
    with_ride_time.insert(loc = 10, column = "month_year", value = extract_month_year(cleaned_data))
    # add total_ridership
    with_ride_time.insert(loc = 11, column = "total_ridership", value = total_ridership(raw_data_df))
    
    return with_ride_time

In [63]:
processed_early_samples_df = pd.DataFrame()

In [64]:
for month in raw_data_early:
    processed_month = process_early_better(month)
    
    processed_early_samples_df = pd.concat(objs = [processed_early_samples_df, processed_month])
    processed_early_samples_df = processed_early_samples_df.reset_index(drop=True)

In [93]:
processed_later_samples_df = pd.DataFrame()

In [94]:
for month in raw_data_later:
    processed_month = process_later_better(month)
    processed_later_samples_df = pd.concat(objs = [processed_later_samples_df, processed_month])
    processed_later_samples_df = processed_later_samples_df.reset_index(drop = True)

In [96]:
total_data_df = pd.concat(objs = [processed_early_samples_df, processed_later_samples_df])

,started_at,ended_at,start_station_name,start_lat,start_lng,end_station_name,end_lat,end_lng,member_casual,ride_time,month_year,total_ridership
0,2018-09-01 00:20:07.6320,2018-09-01 00:37:56.9220,Broadway & W 41 St,40.755,-73.987,W 92 St & Broadway,40.792,-73.974,member,1069,09-2018,1877884
1,2018-09-01 00:23:14.1230,2018-09-01 00:27:45.6990,W 45 St & 6 Ave,40.757,-73.983,Central Park S & 6 Ave,40.766,-73.976,member,271,09-2018,1877884
2,2018-09-01 00:41:27.6280,2018-09-01 00:56:39.5090,Pearl St & Hanover Square,40.705,-74.009,Lafayette St & E 8 St,40.730,-73.991,member,911,09-2018,1877884
3,2018-09-01 00:44:32.2810,2018-09-01 00:56:19.9460,Suffolk St & Stanton St,40.721,-73.985,Broadway & E 14 St,40.735,-73.991,member,707,09-2018,1877884
4,2018-09-01 00:46:39.0610,2018-09-01 00:55:16.6650,Columbus Ave & W 103 St,40.797,-73.964,Adam Clayton Powell Blvd & W 118 St,40.804,-73.951,casual,517,09-2018,1877884
...,...,...,...,...,...,...,...,...,...,...,...,...
99706,2022-09-29 07:51:10,2022-09-29 07:59:01,LaGuardia Pl & W 3 St,40.729,-73.998,Lafayette St & E 8 St,40.730,-73.991,member,471,09-2022,3507123
99707,2022-09-24 14:52:25,2022-09-24 15:01:42,10 Ave & W 14 St,40.742,-74.008,Christopher St & Greenwich St,40.733,-74.007,member,557,09-2022,3507123
99708,2022-09-20 12:11:29,2022-09-20 12:20:34,Broadway & W 25 St,40.743,-73.989,E 22 St & 2 Ave,40.737,-73.981,member,545,09-2022,3507123
99709,2022-09-05 19:11:05,2022-09-05 19:15:53,Pitt St & Stanton St,40.719,-73.982,1 Ave & E 6 St,40.726,-73.986,member,288,09-2022,3507123


In [97]:
total_data_df.to_csv("raw_data/sep_18_to_sep_22.csv")